In [8]:
import numpy as np
import qutip as qt
from scipy.linalg import logm, expm
from qiskit.quantum_info import Operator, state_fidelity
from qiskit import QuantumCircuit
from qiskit_aer import StatevectorSimulator
from qiskit import Aer
from qiskit.circuit.library import QFT


from tools.classical import *
from tools.quantum import *

In [9]:
num_qubits = 3
num_estimating_qubits = 4
qpe_precision = 2 ** (-num_estimating_qubits)

X = qt.sigmax()
Y = qt.sigmay()
Z = qt.sigmaz()

# print(hamiltonian_qt)
#* HAMILTONIANS
# rescaling_factor, shift = rescaling_and_shift_factors(hamiltonian_qt)  # Shift already rescaled
# just_shift = shift * rescaling_factor                                  # Unscaled shift
# shifted_hamiltonian_qt = hamiltonian_qt + just_shift * qt.qeye(hamiltonian_qt.shape[0])
# rescaled_hamiltonian_qt = hamiltonian_qt / rescaling_factor + shift * qt.qeye(hamiltonian_qt.shape[0])

### Find Heisenberg coefficients for non degenerate spectrum

In [10]:
coeff_lower_bound = 0
coeff_xx = np.arange(1, coeff_lower_bound, -0.1)
coeff_yy = np.arange(1, coeff_lower_bound, -0.1)
coeff_zz = np.arange(1, coeff_lower_bound, -0.1)
coeff_z = 1.0

# get all combinations of coefficients
coeffs = np.array(np.meshgrid(coeff_xx, coeff_yy, coeff_zz, coeff_z)).T.reshape(-1, 4)
hamiltonian_ideal_qt = None
coeffs_ideal_spec = []
found_ideal_spectrum = False
for coeff in coeffs:
    hamiltonian_qt = hamiltonian_matrix([X, X], [Y, Y], [Z, Z], coeffs=coeff, num_qubits=num_qubits, symbreak_term=[Z])
    rescaled_hamitlonian_qt = rescale_and_shift_spectrum(hamiltonian_qt)
    
    exact_spec = np.linalg.eigvalsh(hamiltonian_qt)
    print('Original spectrum: ', np.round(exact_spec, 4))
    rescaled_exact_spec = np.linalg.eigvalsh(rescaled_hamitlonian_qt)
    
    # Accept coeff only if all spectrum elements are not closer than qpe_precision
    for eigval_i in rescaled_exact_spec:
        spec_without_eigval_i = np.delete(rescaled_exact_spec, np.where(rescaled_exact_spec == eigval_i))
        if np.any(np.abs(spec_without_eigval_i - eigval_i) < qpe_precision):
            break
    else:
        found_ideal_spectrum = True
        hamiltonian_ideal_qt = hamiltonian_qt
        coeffs_ideal_spec = coeff
        print("Unique spectrum found: ", np.round(rescaled_exact_spec, 4))
        print("Nonrescaled coefficients: ", coeffs_ideal_spec)
        break

if not found_ideal_spectrum:
    print("No ideal spectrum found")
    
#* Found ideal spectrum with bulk sym break term Z, strength 1, for 3-4 and 4-6 system qubits - estimating qubits
    
rescaling_factor, shift = rescaling_and_shift_factors(hamiltonian_ideal_qt)
print(f'Rescaling factor {rescaling_factor}, shift {shift}')
rescaled_hamiltonian_qt = hamiltonian_qt / rescaling_factor + shift * qt.qeye(hamiltonian_qt.shape[0])  #* Shift / No Shift
rescaled_coeff = coeffs_ideal_spec / rescaling_factor
print('Rescaled coefficients: ', rescaled_coeff)


Original spectrum:  [-4.     -3.4641 -2.8284 -2.      2.      2.8284  3.4641  4.    ]
Unique spectrum found:  [-0.      0.067   0.1464  0.25    0.75    0.8536  0.933   1.    ]
Nonrescaled coefficients:  [1. 1. 1. 1.]
Rescaling factor 8.0, shift 0.5
Rescaled coefficients:  [0.125 0.125 0.125 0.125]


In [11]:
exact_spec, eigenstates = np.linalg.eigh(rescaled_hamiltonian_qt)
eig_index = 5
eigenstate = eigenstates[:, eig_index]

initial_exact_energy = exact_spec[eig_index]
initial_eigenstate = Statevector(eigenstate)
print(f'Initial energy = {initial_exact_energy}')

# Initialize
qr_energy = QuantumRegister(num_estimating_qubits, name="E")
cr_energy = ClassicalRegister(num_estimating_qubits, name="crE")
qr_sys  = QuantumRegister(num_qubits, name="sys")
circ = QuantumCircuit(qr_energy, qr_sys, cr_energy)
circ.initialize(initial_eigenstate, qr_sys)

circ.h(qr_energy)

statevector = Statevector(circ).data
padded_rescaled_hamiltonian = np.kron(rescaled_hamiltonian_qt.full(), np.eye(2**num_estimating_qubits))  # top-bottom = right-left
energy = statevector.conj().T @ padded_rescaled_hamiltonian @ statevector
print(energy)

# QPE
T = 1
time = T * 2 * np.pi
num_trotter_steps = 10

trotter_step_circ = trotter_step_heisenberg(num_qubits, coeffs=rescaled_coeff, symbreak=True)
U = ham_evol(num_qubits, trotter_step=trotter_step_circ, num_trotter_steps=num_trotter_steps, time=time)
cU = U.control(1)
for q in range(num_estimating_qubits):
    circ.p(time * shift * 2**q, qr_energy[q])  #* Shift
    for _ in range(2**q):
        circ.compose(cU, [q, *list(qr_sys)], inplace=True)

# Inverse QFT
qft_circ = QFT(num_estimating_qubits, inverse=True)  # With SWAPs
circ.compose(qft_circ, qr_energy, inplace=True)

circ.measure(qr_energy, cr_energy)
print(circ)

Initial energy = 0.8535533905932734
(0.8535533905932727+0j)
                         ┌───┐                    ┌──────┐                  »
  E_0: ──────────────────┤ H ├────────────────────┤ P(π) ├──■───────────────»
                         ├───┤                   ┌┴──────┤  │               »
  E_1: ──────────────────┤ H ├───────────────────┤ P(2π) ├──┼─────■─────■───»
                         ├───┤                   ├───────┤  │     │     │   »
  E_2: ──────────────────┤ H ├───────────────────┤ P(4π) ├──┼─────┼─────┼───»
                         ├───┤                   ├───────┤  │     │     │   »
  E_3: ──────────────────┤ H ├───────────────────┤ P(8π) ├──┼─────┼─────┼───»
       ┌─────────────────┴───┴──────────────────┐└───────┘┌─┴──┐┌─┴──┐┌─┴──┐»
sys_0: ┤0                                       ├─────────┤0   ├┤0   ├┤0   ├»
       │                                        │         │    ││    ││    │»
sys_1: ┤1 Initialize(0,0,0,0.5,0,0.70711,0.5,0) ├─────────┤1 H ├┤1 H ├┤1 H ├»
    

In [12]:
tr_circ = transpile(circ, basis_gates=['u', 'cx'], optimization_level=3)

In [13]:
simulator = Aer.get_backend('statevector_simulator')
shots = 1000
job = simulator.run(tr_circ, shots=shots)
counts = job.result().get_counts()
print(counts)

#! For non rescaled spectrum, under 100 Trotter steps we have garbage as we found out!

{'0110': 5, '1000': 4, '1010': 6, '1001': 4, '0101': 2, '0000': 17, '0010': 11, '1011': 13, '0001': 6, '0011': 1, '1101': 184, '1111': 44, '0111': 3, '1110': 660, '0100': 4, '1100': 36}


In [14]:
phase_bits = max(counts, key=counts.get) # take the most often obtaned result
phase = int(phase_bits, 2) / 2**num_estimating_qubits  # exp(i 2pi phase)
    
estimated_energy = phase / T  # exp(i 2pi phase) = exp(i 2pi E T)
print(f'Exact energy: {initial_exact_energy}')
print(f'Estimated energy: {estimated_energy}')

Exact energy: 0.8535533905932734
Estimated energy: 0.875
